In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
file_path = './MO14-Round-1-Dealing-With-Data-Workbook.xlsx'
sheet_names = ['Contracts', 'Usage']

dfs = pd.read_excel(file_path, sheet_name=sheet_names)

df_contracts = dfs['Contracts']
df_usage = dfs['Usage']

In [3]:
# 1. Usage EDA and Cleaning
df_usage.info()

original_header = df_usage.columns[0]
df_usage.loc[-1] = [original_header] 
df_usage.index = df_usage.index + 1 
df_usage = df_usage.sort_index()
df_usage = df_usage.reset_index(drop=True)
df_usage = df_usage.rename(columns={df_usage.columns[0]: 'dirty_column'})
df_usage.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 1 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0    3 PM  Mon 24th-Mar-2014___0.384 kwh    8759 non-null   object
dtypes: object(1)
memory usage: 68.6+ KB


,dirty_column
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
5,_5AM Tue 19th-Aug-2014___1.228 kwh
6,12PM Mon 7th-Jul-2014___1.296 kwh
7,7 AM Tue 25-Nov-2014__0.437 kwh
8,_8AM 14-Aug-2014__0.523 kwh
9,__4PM 25th-Jan-2014__2.052kwh


In [4]:
# checking if all rows have a string - 'kwh' (standalone)
contains_kwh = df_usage['dirty_column'].str.contains(r'\b(kwh)\b', case=False)
contains_kwh.all()

# checking if all rows have a string - 'kwh' that may be not clean
contains_kwh_notsa = df_usage['dirty_column'].str.contains('kwh', case=False)
contains_kwh_notsa.all()

# checking if all rows have a string - 'am' or 'pm' but clean
contains_am_pm = df_usage['dirty_column'].str.contains(r'\b(am|pm)\b', case=False, regex=True)
contains_am_pm.all()

# checking if all rows have a string - 'am' or 'pm' but may be not clean
contains_am_pm_notsa = df_usage['dirty_column'].str.contains('am|pm', case=False)
contains_am_pm_notsa.all()

True

In [19]:
dirty_electricity = df_usage['dirty_column'].str.extract(r'_(.*?)kwh')
dirty_electricity[0].str.contains('_').all()
dirty_electricity[0].isnull().values.any()
dirty_electricity[0].str.extract(r'(?:_)?(\d+\.\d+)', expand=False).astype(float).isnull().sum()
clean_electricity_column = dirty_electricity[0].str.extract(r'(?:_)?(\d+\.\d+)', expand=False).astype(float)
clean_electricity_column

0       0.384
1       1.201
2       1.523
3       0.424
4       0.209
        ...  
8755    0.084
8756    1.027
8757    0.052
8758    1.428
8759    0.528
Name: 0, Length: 8760, dtype: float64

In [6]:
df_usage['dirty_column'].str.contains(r'(.*?)-(.*?)-(.*?)').all()
df_usage['dirty_column'].str.contains(r'\s(.*?)-(.*?)-(.*?)_').all()
(df_usage['dirty_column'].str.extract(r'\s(.*?)-(.*?)-(.*?)_')[2] == '2014').all()
month_column = df_usage['dirty_column'].str.extract(r'\s(.*?)-(.*?)-(.*?)_')[1]
month_column

0       Mar
1       Aug
2       Mar
3       Apr
4       Dec
       ... 
8755    Nov
8756    May
8757    Apr
8758    Jan
8759    Aug
Name: 1, Length: 8760, dtype: object

In [7]:
df_date_dirty = df_usage['dirty_column'].str.split('-', n=1).str[0]
df_date_dirty
date_list_s = df_date_dirty.str.split()
date_list_s

0            [3, PM, Mon, 24th]
1                     [5AM, 15]
2              [__8PM, Thu, 20]
3                   [6PM, 23rd]
4          [_1AM, Friday, 19th]
                 ...           
8755          [_1AM, Fri, 07th]
8756                 [_6AM, 20]
8757    [__2, AM, Tuesday, 8th]
8758              [9, PM, 27th]
8759            [_12, PM, 25th]
Name: dirty_column, Length: 8760, dtype: object

In [8]:
def extract_digits(text):
    return ''.join(filter(str.isdigit, text))

def extract_weekday(lst):
    lower_list = [item.lower() for item in lst]
    result = [item for item in lower_list if item in {'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'}]
    return result[0] if result else pd.NA

def replace_if_contains_day(text):
    if pd.isna(text):
        return text
    elif 'day' in text.lower(): 
        return text[:3] 
    return text

def extract_am_pm(lst):
    lower_list = [item.lower() for item in lst]
    for item in lower_list:
        if 'am' in item:
            return 'AM'
        elif 'pm' in item:
            return 'PM'
    return pd.NA

In [9]:
hour_column = date_list_s.apply(lambda x: extract_digits(x[0]) if x else '').astype(int)
hour_column.isnull().sum()
hour_column

0        3
1        5
2        8
3        6
4        1
        ..
8755     1
8756     6
8757     2
8758     9
8759    12
Name: dirty_column, Length: 8760, dtype: int32

In [10]:
day_column = date_list_s.apply(lambda x: extract_digits(x[len(x)-1]) if x else '').astype(int)
day_column

0       24
1       15
2       20
3       23
4       19
        ..
8755     7
8756    20
8757     8
8758    27
8759    25
Name: dirty_column, Length: 8760, dtype: int32

In [11]:
weekday_column = date_list_s.apply(extract_weekday)
clean_weekday = weekday_column.apply(replace_if_contains_day)
clean_weekday.head(10)

0     mon
1    <NA>
2     thu
3    <NA>
4     fri
5     tue
6     mon
7     tue
8    <NA>
9    <NA>
Name: dirty_column, dtype: object

In [12]:
time_indicator = date_list_s.apply(extract_am_pm)
time_indicator

0       PM
1       AM
2       PM
3       PM
4       AM
        ..
8755    AM
8756    AM
8757    AM
8758    PM
8759    PM
Name: dirty_column, Length: 8760, dtype: object

In [13]:
df_usage_cleaned = pd.DataFrame({
    'Hour': hour_column,
    'TI(AM/PM)': time_indicator,
    'Weekday': clean_weekday,
    'Day': day_column,
    'Month': month_column,
    'Electricity(kwh)': clean_electricity_column
})

df_usage_cleaned

,Hour,TI(AM/PM),Weekday,Day,Month,Electricity(kwh)
0,3,PM,mon,24,Mar,0.384
1,5,AM,<NA>,15,Aug,1.201
2,8,PM,thu,20,Mar,1.523
3,6,PM,<NA>,23,Apr,0.424
4,1,AM,fri,19,Dec,0.209
...,...,...,...,...,...,...
8755,1,AM,fri,7,Nov,0.084
8756,6,AM,<NA>,20,May,1.027
8757,2,AM,tue,8,Apr,0.052
8758,9,PM,<NA>,27,Jan,1.428


In [15]:
df_usage_cleaned.to_csv('usage.csv', index=False)

In [16]:
# 2. Contracts EDA and Cleaning

df = df_contracts.dropna(how='all').dropna(axis=1, how='all')

def clean_table(table):
    table = table.dropna(how='all').dropna(axis=1, how='all')
    table.columns = table.iloc[0]
    table = table[1:]
    table = table.reset_index(drop=True)
    return table

no_flex = df.iloc[1:4, 0:2]
no_flex_cleaned = clean_table(no_flex)

monthly_flex = df.iloc[1:15, 3:5]
monthly_flex_cleaned = clean_table(monthly_flex)

hourly_flex = df.iloc[1:, 6:8]
hourly_flex_cleaned = clean_table(hourly_flex)

no_flex_cleaned.drop(columns='No Flex', inplace=True)
no_flex_cleaned.columns = ['Rate']
# no_flex_cleaned
hourly_flex_cleaned.columns = ['Hourly', 'Rate']
# hourly_flex_cleaned
monthly_flex_cleaned.columns = ['Month', 'Rate']
# monthly_flex_cleaned

file_name = 'contracts.xlsx'
with pd.ExcelWriter('contracts.xlsx') as writer:
    no_flex_cleaned.to_excel(writer, sheet_name='no flex', index=False)
    hourly_flex_cleaned.to_excel(writer, sheet_name='hourly flex', index=False)
    monthly_flex_cleaned.to_excel(writer, sheet_name='monthly flex', index=False)

In [17]:
#test
df = pd.read_excel('./contracts.xlsx', sheet_name='no flex')
df1 = pd.read_excel('./contracts.xlsx', sheet_name='hourly flex')
df2 = pd.read_excel('./contracts.xlsx', sheet_name='monthly flex')
df2

,Month,Rate
0,January,0.20
1,February,0.19
2,March,0.17
3,April,0.18
4,May,0.22
5,June,0.27
6,July,0.24
7,August,0.19
8,September,0.18
9,October,0.15
